In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
from nemosis import static_table, dynamic_data_compiler
import plotly.express as px

raw_data_cache = '/Volumes/T7/NEMO_data'

In [3]:
# Get the volume bids for the energy market.
dispatch_load = dynamic_data_compiler(start_time='2021/03/01 00:00:00',
                                   end_time='2021/03/10 00:00:00',
                                   table_name='DISPATCHLOAD',
                                   raw_data_location=raw_data_cache)

INFO: Compiling data for table DISPATCHLOAD
INFO: Returning DISPATCHLOAD.


In [4]:
dispatch_load

,SETTLEMENTDATE,DUID,INTERVENTION,DISPATCHMODE,AGCSTATUS,INITIALMW,TOTALCLEARED,RAMPDOWNRATE,RAMPUPRATE,LOWER5MIN,...,RAISE60SEC,RAISE6SEC,LOWERREG,RAISEREG,AVAILABILITY,RAISEREGENABLEMENTMAX,RAISEREGENABLEMENTMIN,LOWERREGENABLEMENTMAX,LOWERREGENABLEMENTMIN,SEMIDISPATCHCAP
0,2021-03-01 00:05:00,AGLHAL,0,0,0,0.00000,0.00000,720.00,720.00,0.0,...,0.0000,0.0000,0.0,0.0,175.00000,0.0,0.0,0.0,0.0,0
1,2021-03-01 00:05:00,AGLSOM,0,0,0,0.00000,0.00000,480.00,480.00,0.0,...,0.0000,0.0000,0.0,0.0,160.00000,0.0,0.0,0.0,0.0,0
2,2021-03-01 00:05:00,ANGAST1,0,0,0,0.00000,0.00000,840.00,840.00,0.0,...,0.0000,0.0000,0.0,0.0,40.00000,0.0,0.0,0.0,0.0,0
3,2021-03-01 00:05:00,APD01,0,0,0,0.00000,0.00000,0.00,0.00,0.0,...,0.0001,0.0001,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0
4,2021-03-01 00:05:00,ARWF1,0,0,0,162.50000,161.88836,600.00,1200.00,0.0,...,0.0000,0.0000,0.0,0.0,161.88836,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914971,2021-03-10 00:00:00,YENDWF1,0,0,0,12.41000,12.02600,1680.00,1680.00,0.0,...,0.0000,0.0000,0.0,0.0,12.02600,0.0,0.0,0.0,0.0,0
914972,2021-03-10 00:00:00,YWPS1,0,0,1,379.44809,380.00000,180.00,180.00,0.0,...,0.0000,0.0000,0.0,0.0,380.00000,365.0,250.0,365.0,250.0,0
914973,2021-03-10 00:00:00,YWPS2,0,0,1,338.84558,340.00000,180.00,180.00,10.0,...,0.0000,0.0000,0.0,0.0,340.00000,355.0,250.0,355.0,250.0,0
914974,2021-03-10 00:00:00,YWPS3,0,0,1,372.93799,375.00000,172.13,172.13,15.0,...,0.0000,0.0000,0.0,0.0,375.00000,385.0,250.0,385.0,250.0,0


In [5]:
# Group by DUID and sum the TOTALCLEARED column
generation_ranking = dispatch_load.groupby('DUID')['TOTALCLEARED'].sum().sort_values(ascending=False)

# Convert to dataframe and add rank column
generation_ranking = generation_ranking.reset_index()
generation_ranking['rank'] = generation_ranking.index + 1

# Show top 10
print("Top 10 generators by total dispatch:")
print(generation_ranking.head(10))

Top 10 generators by total dispatch:
     DUID  TOTALCLEARED  rank
0   KPP_1  1.866240e+06     1
1  LOYYB1  1.418682e+06     2
2  LOYYB2  1.385271e+06     3
3     MP1  1.360302e+06     4
4     MP2  1.339131e+06     5
5    LYA3  1.336849e+06     6
6    LYA1  1.336133e+06     7
7    LYA4  1.329320e+06     8
8    BW03  1.327687e+06     9
9    LYA2  1.305249e+06    10


In [9]:
# Download the latest Generators and Scheduled Loads table. The 
# update_static_file=True argument forces nemosis to download a new copy of 
# file from AEMO even if a copy already exists in the cache.
dispatch_units = static_table(table_name='Generators and Scheduled Loads', 
                              raw_data_location=raw_data_cache,
                              update_static_file=False)
dispatch_units

INFO: Retrieving static table Generators and Scheduled Loads


,Participant,Station Name,Region,Dispatch Type,Category,Classification,Fuel Source - Primary,Fuel Source - Descriptor,Technology Type - Primary,Technology Type - Descriptor,Aggregation,DUID
0,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,ADPBA1G
1,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Load,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,ADPBA1L
2,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Non-Scheduled,Hydro,Water,Renewable,Run of River,Y,ADPMH1
3,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Semi-Scheduled,Solar,Solar,Renewable,Photovoltaic Tracking Flat panel,Y,ADPPV1
4,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Non-Scheduled,Solar,Solar,Renewable,Photovoltaic Flat panel,Y,ADPPV2
...,...,...,...,...,...,...,...,...,...,...,...,...
527,Tailem Bend II Project Company Pty Ltd as trus...,Tailem Bend 2 Hybrid Renewable Power Station,SA1,Bidirectional Unit,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,TB2B1
528,AGL Macquarie Pty Limited,Broken Hill Battery Energy Storage System,NSW1,Bidirectional Unit,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,BHB1
529,AGL SA Generation Pty Limited,Torrens Island BESS,SA1,Bidirectional Unit,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,TIB1
530,Capital Battery Pty Ltd as Trustee for Capital...,Capital Battery,NSW1,Load,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,CAPBES1


In [7]:
# First get our ranking as before
generation_ranking = dispatch_load.groupby('DUID')['TOTALCLEARED'].sum().sort_values(ascending=False)
generation_ranking = generation_ranking.reset_index()
generation_ranking['rank'] = generation_ranking.index + 1

# Merge with dispatch_units table to get additional information
generator_details = generation_ranking.merge(
    dispatch_units[['DUID', 'Station Name', 'Region', 'Fuel Source - Primary', 'Fuel Source - Descriptor']], 
    on='DUID',
    how='left'
)

# Show top 10 with details
print("Top 10 generators by total dispatch with details:")
print(generator_details.head(10).to_string())

Top 10 generators by total dispatch with details:
     DUID  TOTALCLEARED  rank               Station Name Region Fuel Source - Primary Fuel Source - Descriptor
0   KPP_1  1.866240e+06     1  Kogan Creek Power Station   QLD1                Fossil               Black Coal
1  LOYYB1  1.418682e+06     2   Loy Yang B Power Station   VIC1                Fossil               Brown Coal
2  LOYYB2  1.385271e+06     3   Loy Yang B Power Station   VIC1                Fossil               Brown Coal
3     MP1  1.360302e+06     4     Mt Piper Power Station   NSW1                Fossil               Black Coal
4     MP2  1.339131e+06     5     Mt Piper Power Station   NSW1                Fossil               Black Coal
5    LYA3  1.336849e+06     6   Loy Yang A Power Station   VIC1                Fossil               Brown Coal
6    LYA1  1.336133e+06     7   Loy Yang A Power Station   VIC1                Fossil               Brown Coal
7    LYA4  1.329320e+06     8   Loy Yang A Power Station   VIC

Overall steps:
1. Find MC estimates
2. Group units together by the firms that control them, including the firms's daily declared capacity
3. Create a total marginal cost function which represents the cost curve for all of a firm's generating units, stacked from lowest to highest cost. This creates a stepwise increasing function where: X-axis is cumulative MW across all units, Y-axis is marginal cost ($/MWh), each step represents a different generating unit. Width of step = unit's capacity. Height of step = unit's marginal cost
4. Taking only units that are verified to be "on-line" and operating during that hour
5. Subtracting the day-ahead scheduled quantity to center the function around 0
6. Including only natural gas and coal units that can respond quickly (excluding nuclear, wind, hydro)

#Step 1: MC estimates
Black coal: ~$41-101/MWh
Brown coal: ~$12-13/MWh
Wind/Solar: $0-1/MWh

Methodology:
1. Fuel cost range: 
2. Heat rate
3. Variable O&M (Operations and Maintenance)

Note: 
1. This includes a big assumption that MC is the same across every firm for each fuel type, which is not true!
2. For coal, it would be good to include a shutdown cost - I don't want to arbitrarily limit it like Hortaçsu as emperically coal firms are choosing to shut down

In [13]:
# Let's start with Origin Energy Electricity Limited!
# Join the dispatch_units and the dispatch_load tables on DUID

# Perform an outer join to ensure we keep all DUIDs and settlement dates
merged_df = pd.merge(dispatch_load[['SETTLEMENTDATE', 'DUID']], dispatch_units, on="DUID", how="outer")

# Now merge with the full dispatch_load dataset to bring all the fields together
merged_df = pd.merge(merged_df, dispatch_load, on=["DUID", "SETTLEMENTDATE"], how="outer")

# Filter for 'Origin Energy Electricity Limited'
filtered_df = merged_df[merged_df["Participant"] == "Origin Energy Electricity Limited"]

filtered_df

,SETTLEMENTDATE,DUID,Participant,Station Name,Region,Dispatch Type,Category,Classification,Fuel Source - Primary,Fuel Source - Descriptor,...,RAISE60SEC,RAISE6SEC,LOWERREG,RAISEREG,AVAILABILITY,RAISEREGENABLEMENTMAX,RAISEREGENABLEMENTMIN,LOWERREGENABLEMENTMAX,LOWERREGENABLEMENTMIN,SEMIDISPATCHCAP
202238,2021-03-01 00:05:00,DDPS1,Origin Energy Electricity Limited,Darling Downs Power Station,QLD1,Generating Unit,Market,Scheduled,Fossil,Natural Gas,...,10.0,10.0,10.0,10.0,210.0,154.28358,110.0,154.28358,110.0,0.0
202239,2021-03-01 00:10:00,DDPS1,Origin Energy Electricity Limited,Darling Downs Power Station,QLD1,Generating Unit,Market,Scheduled,Fossil,Natural Gas,...,10.0,10.0,10.0,10.0,210.0,153.83496,110.0,153.83496,110.0,0.0
202240,2021-03-01 00:15:00,DDPS1,Origin Energy Electricity Limited,Darling Downs Power Station,QLD1,Generating Unit,Market,Scheduled,Fossil,Natural Gas,...,10.0,10.0,10.0,10.0,210.0,156.86850,110.0,156.86850,110.0,0.0
202241,2021-03-01 00:20:00,DDPS1,Origin Energy Electricity Limited,Darling Downs Power Station,QLD1,Generating Unit,Market,Scheduled,Fossil,Natural Gas,...,10.0,10.0,10.0,10.0,210.0,153.04453,110.0,153.04453,110.0,0.0
202242,2021-03-01 00:25:00,DDPS1,Origin Energy Electricity Limited,Darling Downs Power Station,QLD1,Generating Unit,Market,Scheduled,Fossil,Natural Gas,...,10.0,10.0,10.0,10.0,210.0,154.24086,110.0,154.24086,110.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808900,2021-03-09 23:45:00,URANQ14,Origin Energy Electricity Limited,Uranquinty Power Station,NSW1,Generating Unit,Market,Scheduled,Fossil,Natural Gas,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
808901,2021-03-09 23:50:00,URANQ14,Origin Energy Electricity Limited,Uranquinty Power Station,NSW1,Generating Unit,Market,Scheduled,Fossil,Natural Gas,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
808902,2021-03-09 23:55:00,URANQ14,Origin Energy Electricity Limited,Uranquinty Power Station,NSW1,Generating Unit,Market,Scheduled,Fossil,Natural Gas,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
808903,2021-03-10 00:00:00,URANQ14,Origin Energy Electricity Limited,Uranquinty Power Station,NSW1,Generating Unit,Market,Scheduled,Fossil,Natural Gas,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0


Note Origin's percentage of bids in the time period of NEMO was 67392/914976 = 7.37%. So it actually seems like the dominant firm Origin might not have that much market share, which lends itself to perfect competition hypothesis that P=MC 